#### 1.2 starts a spark session

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('excercise').getOrCreate()

#### 1.3 load data

In [0]:
path = '/FileStore/tables/walmart_stock.csv'
df = spark.read.csv(path, inferSchema = True, header = True)

#### 1.4 column names

In [0]:
df.columns

Out[6]: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### 1.5 schema

In [0]:
df.printSchema()

root
-- Date: string (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: integer (nullable = true)
-- Adj Close: double (nullable = true)

#### 1.6 print out the first 5 rows

In [0]:
for i in df.head(5): print(i)

Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)
Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)
Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)
Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)
Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)

#### 1.7 describe df

In [0]:
df.describe().drop(col('Date')).show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
summary| Open| High| Low| Close| Volume| Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
 count| 1258| 1258| 1258| 1258| 1258| 1258|
 mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
 stddev| 6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991| 4519780.8431556|6.722609449996857|
 min|56.389998999999996| 57.060001| 56.299999| 56.419998| 2094900| 50.363689|
 max| 90.800003| 90.970001| 89.25| 90.470001| 80898100|84.91421600000001|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+

####2.1 format meana dn stddev numbers in describe to 2 decimal places

In [0]:
des = df.describe().drop(col('Date'))

In [0]:
from pyspark.sql.functions import format_number, col
from pyspark.sql.types import DoubleType, IntegerType

In [0]:
des.printSchema()

root
-- summary: string (nullable = true)
-- Open: string (nullable = true)
-- High: string (nullable = true)
-- Low: string (nullable = true)
-- Close: string (nullable = true)
-- Volume: string (nullable = true)
-- Adj Close: string (nullable = true)

In [0]:
des = des.withColumn('Open', des.Open.cast(DoubleType()))\
.withColumn('High', des.High.cast(DoubleType()))\
.withColumn('Low', des.Low.cast(DoubleType()))\
.withColumn('Close', des.Close.cast(DoubleType()))\
.withColumn('Volume', des.Volume.cast(IntegerType()))\
.withColumn('Adj Close', des['Adj Close'].cast(DoubleType()))

In [0]:
des.select('summary', format_number(des['Open'], 2).alias('Open'),format_number(des['High'], 2).alias('High'),format_number(des['Low'], 2).alias('Low'),format_number(des['Close'], 2).alias('Close'), 'Volume').show()

+-------+--------+--------+--------+--------+--------+
summary| Open| High| Low| Close| Volume|
+-------+--------+--------+--------+--------+--------+
 count|1,258.00|1,258.00|1,258.00|1,258.00| 1258|
 mean| 72.36| 72.84| 71.92| 72.39| 8222093|
 stddev| 6.77| 6.77| 6.74| 6.76| 4519780|
 min| 56.39| 57.06| 56.30| 56.42| 2094900|
 max| 90.80| 90.97| 89.25| 90.47|80898100|
+-------+--------+--------+--------+--------+--------+

In [0]:
# 2nd solution: more concise, directly cast it
result = df.describe().drop(col('Date'))
result.select('summary', 
           format_number(result['Open'].cast('float'), 2).alias('Open'),
              format_number(result['High'].cast('float'), 2).alias('High'),
              format_number(result['Low'].cast('float'), 2).alias('Low'),
              format_number(result['Close'].cast('float'), 2).alias('Close'), 
              result['Volume'].cast('int').alias('Volume')
             ).show()

+-------+--------+--------+--------+--------+--------+
summary| Open| High| Low| Close| Volume|
+-------+--------+--------+--------+--------+--------+
 count|1,258.00|1,258.00|1,258.00|1,258.00| 1258|
 mean| 72.36| 72.84| 71.92| 72.39| 8222093|
 stddev| 6.77| 6.77| 6.74| 6.76| 4519780|
 min| 56.39| 57.06| 56.30| 56.42| 2094900|
 max| 90.80| 90.97| 89.25| 90.47|80898100|
+-------+--------+--------+--------+--------+--------+

#### 2.2 create new dataframe with a column called HV ratio (High/volumn)

In [0]:
df.select((df['High']/df['Volume']).alias('HV Ratio')).show()

+--------------------+
 HV Ratio|
+--------------------+
4.819714653321546E-6|
6.290848613094555E-6|
4.669412994783916E-6|
7.367338463826307E-6|
8.915604778943901E-6|
8.644477436914568E-6|
9.351828421515645E-6|
 8.29141562102703E-6|
7.712212102001476E-6|
7.071764823529412E-6|
1.015495466386981E-5|
6.576354146362592...|
 5.90145296180676E-6|
8.547679455011844E-6|
8.420709512685392E-6|
1.041448341728929...|
8.316075414862431E-6|
9.721183814992126E-6|
8.029436027707578E-6|
6.307432259386365E-6|
+--------------------+
only showing top 20 rows

In [0]:
#2nd solution:
df2 = df.withColumn('HV Ratio', df['High']/df['Volume'])
df2.select('HV Ratio').show()

+--------------------+
 HV Ratio|
+--------------------+
4.819714653321546E-6|
6.290848613094555E-6|
4.669412994783916E-6|
7.367338463826307E-6|
8.915604778943901E-6|
8.644477436914568E-6|
9.351828421515645E-6|
 8.29141562102703E-6|
7.712212102001476E-6|
7.071764823529412E-6|
1.015495466386981E-5|
6.576354146362592...|
 5.90145296180676E-6|
8.547679455011844E-6|
8.420709512685392E-6|
1.041448341728929...|
8.316075414862431E-6|
9.721183814992126E-6|
8.029436027707578E-6|
6.307432259386365E-6|
+--------------------+
only showing top 20 rows

#### 2.3 what day had the peak high price

In [0]:
df.orderBy(df['High'].desc()).select('Date').head()[0]

Out[98]: '2015-01-13'

In [0]:
df.orderBy(df['High'].desc()).head()[0]

Out[97]: '2015-01-13'

#### 2.4 what is the mean of close column

In [0]:
from pyspark.sql import functions as f

In [0]:
df.select(f.avg('Close')).show()

+-----------------+
 avg(Close)|
+-----------------+
72.38844998012726|
+-----------------+

In [0]:
#2nd solution:
from pyspark.sql.functions import mean, max, min
df.select(mean('Close')).show()

+-----------------+
 avg(Close)|
+-----------------+
72.38844998012726|
+-----------------+

#### 2.5 what is the max and min of Volumn column

In [0]:
df.select(f.max('Volume'), f.min('volume')).show()

+-----------+-----------+
max(Volume)|min(volume)|
+-----------+-----------+
 80898100| 2094900|
+-----------+-----------+

In [0]:
#2nd solution:
df.select(max('Volume'), min('Volume')).show()

+-----------+-----------+
max(Volume)|min(Volume)|
+-----------+-----------+
 80898100| 2094900|
+-----------+-----------+

#### 2.6 how many days was the close lower than 60 dollars?

In [0]:
df.groupBy(df['Close']<60).count().select('count').collect()[0]['count']

Out[75]: 81

In [0]:
#2n solution:
df.filter(df['Close']<60).count() # df.select('Close<60').count()

Out[104]: 81

In [0]:
#3rd solution:
from pyspark.sql.functions import count
result = df.filter(df['Close']<60)
result.select(count('Close')).show()

+------------+
count(Close)|
+------------+
 81|
+------------+

#### 2.7 + 2.8 what percentage of the time was the high greater than 80 dollars?

In [0]:
(df.groupBy(df['High']>80).count().select('count').collect()[0]['count']/df.count())*100

Out[79]: 9.141494435612083

In [0]:
df.filter(df['High']>80).count()/df.count()*100

Out[107]: 9.141494435612083

#### 2.9+ 2.10 pearson correlation between High and Volume

In [0]:
df.select(f.corr('High','Volume')).show()

+-------------------+
 corr(High, Volume)|
+-------------------+
-0.3384326061737161|
+-------------------+

In [0]:
df.stat.corr('High', 'Volume')

Out[80]: -0.3384326061737161

#### 2.11 what is the max high per year?

In [0]:
from pyspark.sql.functions import max, year, month, avg

In [0]:
df.groupBy(year('Date').alias('Year')).max('High').show()

+----+---------+
Year|max(High)|
+----+---------+
2015|90.970001|
2013|81.370003|
2014|88.089996|
2012|77.599998|
2016|75.190002|
+----+---------+

#### 2.12 + 2.13 what is the average close for each calender month

In [0]:
df.groupBy(month('Date').alias('Month')).avg('Close').orderBy('Month').show()

+-----+-----------------+
Month| avg(Close)|
+-----+-----------------+
 1|71.44801958415842|
 2| 71.306804443299|
 3|71.77794377570092|
 4|72.97361900952382|
 5|72.30971688679247|
 6| 72.4953774245283|
 7|74.43971943925233|
 8|73.02981855454546|
 9|72.18411785294116|
 10|71.57854545454543|
 11| 72.1110893069307|
 12|72.84792478301885|
+-----+-----------------+